## Dependencies

In [1]:
# Dependencies
import os
import json
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

# Facebook Credentials
FB_EMAIL = "arnab@itobuz.com"  # Replace with your Facebook email
FB_PASSWORD = "Arnab@2002"     # Replace with your Facebook password
# Facebook Page URL
PAGE_URL = "https://www.facebook.com/Itobuz/"  # Replace with the target Facebook page URL
# Folder to save posts and images
SAVE_FOLDER = "facebook_scraped_data"
os.makedirs(SAVE_FOLDER, exist_ok=True)

# Set up Chrome WebDriver using `webdriver-manager`
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Uncomment this line to enable headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36")

# Auto-download the correct ChromeDriver version
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

def facebook_login():
    """
    Logs into Facebook using provided credentials.
    """
    try:
        driver.get("https://www.facebook.com/login")
        time.sleep(5)  # Add a delay to ensure the page loads
        print("Page source after navigating to login:", driver.page_source[:500])  # Debugging
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "email")))
        # Find and fill in the email field
        email_element = driver.find_element(By.ID, "email")
        email_element.send_keys(FB_EMAIL)
        # Find and fill in the password field
        password_element = driver.find_element(By.ID, "pass")
        password_element.send_keys(FB_PASSWORD)
        password_element.send_keys(Keys.RETURN)
        # Wait for login to complete by checking for a known element on the homepage
        WebDriverWait(driver, 20).until(EC.url_contains("facebook.com/home"))
        print("Logged into Facebook successfully.")
    except Exception as e:
        print(f"Login failed: {e}")
        import traceback
        traceback.print_exc()
        driver.quit()
        exit()

def download_image(image_url, post_id, image_index):
    """
    Downloads and saves an image from a given URL.
    """
    try:
        # Skip invalid URLs (e.g., data:image/svg+xml)
        if image_url.startswith("data:"):
            print(f"Skipping invalid image URL: {image_url}")
            return None
        # Ensure the URL is a complete URL
        if image_url.startswith('/'):
            image_url = "https://www.facebook.com" + image_url
        # Make the request to download the image
        response = requests.get(image_url, stream=True)
        if response.status_code == 200:
            image_path = os.path.join(SAVE_FOLDER, f"{post_id}_{image_index}.jpg")
            with open(image_path, "wb") as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            return image_path
        else:
            print(f"Failed to download image. HTTP Status Code: {response.status_code}")
    except Exception as e:
        print(f"Failed to download image: {e}")
    return None

def scroll_page():
    """
    Scrolls down the page to load more posts dynamically.
    """
    SCROLL_PAUSE_TIME = 5
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        print("Scrolling down...")
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("No more content to load.")
            break
        last_height = new_height

def click_see_more_buttons():
    """
    Clicks all 'See more' buttons to reveal full post text.
    """
    try:
        see_more_buttons = driver.find_elements(By.XPATH, "//div[contains(text(), 'See more')]")
        for button in see_more_buttons:
            retries = 3
            for _ in range(retries):
                try:
                    driver.execute_script("arguments[0].scrollIntoView();", button)
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(1)
                    break
                except Exception as e:
                    print(f"Retrying 'See more' click: {e}")
    except Exception as e:
        print(f"Error finding 'See more' buttons: {e}")

def extract_hashtags(post):
    """
    Extracts hashtags from the post by searching for anchor tags with href attributes starting with '/hashtag/'.
    """
    hashtag_links = post.find_all("a", href=lambda href: href and href.startswith("/hashtag/"))
    hashtags = [link.get_text(strip=True) for link in hashtag_links]
    return hashtags

def extract_emojis(post):
    """
    Extracts emojis from the post by filtering <img> tags with emoji-specific attributes.
    """
    emoji_images = post.find_all("img", {"class": "x1b0d499"})  # Update class name if necessary
    emojis = [img["alt"] for img in emoji_images if "alt" in img.attrs]
    return emojis

def extract_post_data(post, index):
    """
    Extracts data from a single post.
    """
    try:
        # Extract post heading
        heading_element = post.find("h3")
        post_heading = heading_element.get_text(strip=True) if heading_element else "No heading available"
        
        # Extract post text
        text_element = post.find("div", {"data-ad-preview": "message"})
        post_text = text_element.get_text(strip=True) if text_element else "No text available"
        
        # Extract hashtags
        hashtags = extract_hashtags(post)
        
        # Extract emojis
        emojis = extract_emojis(post)
        
        # Extract image URLs (excluding emojis and invalid URLs)
        image_elements = post.find_all("img")
        image_urls = [
            img["src"] for img in image_elements 
            if "src" in img.attrs and not img["src"].startswith("data:") and "alt" not in img.attrs
        ]
        local_image_paths = []
        # Download images and save local paths
        for img_index, image_url in enumerate(image_urls):
            local_path = download_image(image_url, index, img_index)
            if local_path:
                local_image_paths.append(local_path)
        
        return {
            "heading": post_heading,
            "text": post_text,
            "hashtags": hashtags,
            "emojis": emojis,
            "image_urls": image_urls,
            "local_image_paths": local_image_paths if local_image_paths else ["No images"]
        }
    except Exception as e:
        print(f"Error extracting post {index}: {e}")
        import traceback
        traceback.print_exc()
        return None

def scrape_facebook_posts():
    """
    Scrapes posts and their images from a Facebook page after login.
    """
    try:
        driver.get(PAGE_URL)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "article")))
        print("Page loaded successfully.")
        
        print("Scrolling to load more posts...")
        scroll_page()
        
        print("Clicking 'See more' buttons...")
        click_see_more_buttons()
        
        print("Extracting posts...")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        posts = soup.find_all("div", {"role": "article"})
        if not posts:
            print("No posts found. Check selectors.")
            return
        
        posts_data = []
        for index, post in enumerate(posts):
            post_data = extract_post_data(post, index)
            if post_data:
                posts_data.append(post_data)
        
        # Save posts to a JSON file
        json_path = os.path.join(SAVE_FOLDER, "facebook_scraped_posts.json")
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(posts_data, f, indent=4)
        print(f"Posts saved successfully in {json_path}")
    except Exception as e:
        print(f"An error occurred during scraping: {e}")
        import traceback
        traceback.print_exc()
    finally:
        driver.quit()

# Run the script
facebook_login()
scrape_facebook_posts()

Page source after navigating to login: <html lang="en" id="facebook" class=""><head><meta charset="utf-8"><meta name="referrer" content="origin-when-crossorigin" id="meta_referrer"><script nonce="">function envFlush(a){function b(b){for(var c in a)b[c]=a[c]}window.requireLazy?window.requireLazy(["Env"],b):(window.Env=window.Env||{},b(window.Env))}envFlush({"useTrustedTypes":false,"isTrustedTypesReportOnly":false,"ajaxpipe_token":"AXi7IGsvFxgBg3m0HiY","stack_trace_limit":30,"timesliceBufferSize":5000,"show_invariant_decoder":false,"co
Login failed: Message: 

An error occurred during scraping: HTTPConnectionPool(host='localhost', port=64766): Max retries exceeded with url: /session/10731433bc90d35f940c247ec191e8eb/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10dcd0d90>: Failed to establish a new connection: [Errno 61] Connection refused'))


Traceback (most recent call last):
  File "/var/folders/jk/wxfv5xn16_b00bdt6v49v7640000gn/T/ipykernel_3444/1969340579.py", line 55, in facebook_login
    WebDriverWait(driver, 20).until(EC.url_contains("facebook.com/home"))
  File "/Users/itobuz/.conda/envs/final_project_env/lib/python3.10/site-packages/selenium/webdriver/support/wait.py", line 146, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 

Traceback (most recent call last):
  File "/Users/itobuz/.conda/envs/final_project_env/lib/python3.10/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/Users/itobuz/.conda/envs/final_project_env/lib/python3.10/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/Users/itobuz/.conda/envs/final_project_env/lib/python3.10/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
Connecti